# Datas to be saved


In [ ]:
import numpy as np

s1 = np.array([[1,-1], [1,-1]])
s2 = np.array([[1,1], [1,1]])

S = (s1, s2)

# NetWork
Calculating weight and predict over the inputs

In [ ]:
def calc_weight(datas):
  data_shape = datas[0].shape
  size = np.nanprod(data_shape)
  d = []
  for i in range(len(datas)):
    d.append(datas[i].reshape(1,size))
  
  W = np.zeros((size,size))

  for i in range(len(datas)):
    W += d[i] * d[i].T
  return W

In [ ]:
def activation(input):
  return np.where(input>0, 1, -1)

In [ ]:
def prediction(input, weight):
  size = np.nanprod(input.shape)
  pred = input.reshape((1,size))
  pred = activation(pred.dot(weight))

  return pred.reshape(input.shape)

In [ ]:
weight = calc_weight(S)
weight

array([[2., 0., 2., 0.],
       [0., 2., 0., 2.],
       [2., 0., 2., 0.],
       [0., 2., 0., 2.]])

In [ ]:
def check(datas, weight, output, verbose=True):
  cnt = 0
  for i in range(len(datas)):
    pred = prediction(datas[i], weight)
    if np.all(output[i] == pred):
      if verbose:
        print("Pattern %d is correctly saved : " %i)
        print("S[%d] = " %i, output[i])
        print("\r")
      cnt+=1
    else:
      if verbose:
        print("Pattern %d is NOT correctly saved" %i)
        print("S[%d] = " %i, output[i])
        print("prediction = ", pred)
        print("\r")
  return cnt

In [ ]:
check(S, weight, S)

Pattern 0 is correctly saved : 
S[0] =  [[ 1 -1]
 [ 1 -1]]

Pattern 1 is correctly saved : 
S[1] =  [[1 1]
 [1 1]]



2

# Noise
making inputs a little bit noisy


In [ ]:
import random
from copy import deepcopy

def noisy(datas, n):
  length = len(datas)
  shape = datas[0].shape
  shape_prd = np.nanprod(shape)
  lst = list(range(shape_prd))
  new_datas = []
  for i in range(length):
    indx = random.sample(lst, n)
    in_ = deepcopy(datas[i].reshape((shape_prd,)))
    in_[indx] = in_[indx] * (-1)
    new_datas.append(in_.reshape(shape))
  return new_datas

In [ ]:
new_S = noisy(S, 1)
print("new S = ", new_S)
check(new_S, weight, S)

new S =  [array([[ 1, -1],
       [ 1,  1]]), array([[-1,  1],
       [ 1,  1]])]
Pattern 0 is correctly saved : 
S[0] =  [[ 1 -1]
 [ 1 -1]]

Pattern 1 is NOT correctly saved
S[1] =  [[1 1]
 [1 1]]
prediction =  [[-1  1]
 [-1  1]]



1

In [ ]:
new_S = noisy(S, 2)
print("new S", new_S)
check(new_S, weight, S)

new S [array([[-1, -1],
       [-1, -1]]), array([[ 1, -1],
       [-1,  1]])]
Pattern 0 is NOT correctly saved
S[0] =  [[ 1 -1]
 [ 1 -1]]
prediction =  [[-1 -1]
 [-1 -1]]

Pattern 1 is NOT correctly saved
S[1] =  [[1 1]
 [1 1]]
prediction =  [[-1 -1]
 [-1 -1]]



0

# Lose
losing some of the information

In [ ]:
def losing(datas, n):
  length = len(datas)
  shape = datas[0].shape
  shape_prd = np.nanprod(shape)
  lst = list(range(shape_prd))
  new_datas = []
  for i in range(length):
    indx = random.sample(lst, n)
    in_ = deepcopy(datas[i].reshape((shape_prd,)))
    in_[indx] = in_[indx] * 0
    new_datas.append(in_.reshape(shape))
  return new_datas

In [ ]:
new_S = losing(S, 1)
print("new S = ", new_S)
check(new_S, weight, S)

new S =  [array([[ 1, -1],
       [ 0, -1]]), array([[1, 1],
       [1, 0]])]
Pattern 0 is correctly saved : 
S[0] =  [[ 1 -1]
 [ 1 -1]]

Pattern 1 is correctly saved : 
S[1] =  [[1 1]
 [1 1]]



2

In [ ]:
new_S = losing(S, 2)
print("new S = ", new_S)
check(new_S, weight, S)

new S =  [array([[ 1,  0],
       [ 0, -1]]), array([[0, 1],
       [1, 0]])]
Pattern 0 is correctly saved : 
S[0] =  [[ 1 -1]
 [ 1 -1]]

Pattern 1 is correctly saved : 
S[1] =  [[1 1]
 [1 1]]



2

# Accuracy

In [ ]:
def acc(inputs, outputs, weight, n, noise=False, loss=False):
  cnt = []
  cnt_ = 0
  if noise:
    for i in range(1000):
      cnt_ = 0
      new_inputs = noisy(inputs, n)
      for j in range(len(inputs)):
        prd = prediction(new_inputs[j], weight)
        if np.all(prd == outputs[j]):
          cnt_+=1
      if cnt_ == 2:
        cnt.append(2)
      else:
        cnt.append(0)
  if loss:
    for i in range(1000):
      cnt_ = 0
      new_inputs = losing(inputs, n)
      for j in range(len(inputs)):
        prd = prediction(new_inputs[j], weight)
        if np.all(prd == outputs[j]):
          cnt_+=1
      if cnt_ == 2:
        cnt.append(2)
      else:
        cnt.append(0)

  return  round(100 * np.count_nonzero(np.array(cnt))/1000 , 2), cnt

In [ ]:
acc(S,S,weight, 1, noise=True)[0], acc(S,S,weight, 2, noise=True)[0], acc(S,S,weight, 1, loss=True)[0], acc(S,S,weight, 2, loss=True)[0]

(0.0, 0.0, 100.0, 54.8)

In [ ]:
def iterate(weight, outputs, num, lose=False, noise=False, n=1000):
  cnt = []
  new = 0
  for i in range(n):
    if lose:
      new = losing(outputs, num)
      cnt.append(check(new, weight, outputs, verbose=False))
    elif noise:
      new = noisy(outputs, num)
      cnt.append(check(new, weight, outputs, verbose=False))

  return 100 * round(np.mean(cnt)/len(outputs), 2)

In [ ]:
iterate(weight, S, 1, noise=True), iterate(weight, S, 2, noise=True)

(25.0, 0.0)

In [ ]:
iterate(weight, S, 1, lose=True), iterate(weight, S, 2, lose=True)

(100.0, 75.0)